In [7]:
!pip3 install mecab-python3

In [8]:
!pip freeze -l

appnope==0.1.2
argon2-cffi==21.2.0
argon2-cffi-bindings==21.2.0
attrs==21.2.0
backcall==0.2.0
bleach==4.1.0
certifi==2021.10.8
cffi==1.15.0
debugpy==1.5.1
decorator==5.1.0
defusedxml==0.7.1
entrypoints==0.3
importlib-resources==5.4.0
ipykernel==6.6.0
ipython==7.30.1
ipython-genutils==0.2.0
ipywidgets==7.6.5
jedi==0.18.1
Jinja2==3.0.3
joblib==1.1.0
jsonschema==4.2.1
jupyter==1.0.0
jupyter-client==7.1.0
jupyter-console==6.4.0
jupyter-core==4.9.1
jupyterlab-pygments==0.1.2
jupyterlab-widgets==1.0.2
krwordrank==1.0.3
MarkupSafe==2.0.1
matplotlib-inline==0.1.3
mecab-python3==1.0.4
mistune==0.8.4
nbclient==0.5.9
nbconvert==6.3.0
nbformat==5.1.3
nest-asyncio==1.5.4
networkx==2.6.3
notebook==6.4.6
numpy==1.21.4
packaging==21.3
pandas==1.3.4
pandocfilters==1.5.0
parso==0.8.3
pexpect==4.8.0
pickleshare==0.7.5
prometheus-client==0.12.0
prompt-toolkit==3.0.24
ptyprocess==0.7.0
pycparser==2.21
Pygments==2.10.0
pyparsing==3.0.6
pyrsistent==0.18.0
python-dateutil==2.8.2
pytz==2021.3
pyzmq==22.3.0
qtc

# Setup

## Packages

In [2]:
from konlpy.tag import Mecab

In [3]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import requests
from requests.auth import HTTPBasicAuth
import gzip
import json
import boto3
import pandas as pd
import os
import numpy as np
from json import JSONDecodeError
from pandas import json_normalize
import matplotlib.pyplot as plt
from krwordrank.word import KRWordRank
import time

## Functions

In [4]:
# path값
# naver 종목토론실 : '/poc_jongtoe/_search?size=10000'
# naver 뉴스 : '/poc_news_agg/_search?size=10000'
# naver 뉴스 기사제목 : '/poc_title_text/_search?size=10000'

def get_es_df(input_date, path, 
              base_url = 'https://search-kis-dp-dev-octoparse-dev-rgpvg4kvlmpyjfzedmfqjpht3a.ap-northeast-2.es.amazonaws.com',
              auth = HTTPBasicAuth('octoparse', 'Octoparse1!'),
              headers = {'Content-Type': 'application/json'}):
    query = {
      "query": { 
          "match": { 
              "input_date" : input_date } }
    }

    query = json.dumps(query).encode()
    print('query: ', query)

    # path = '/poc_title_text/_search?size=10000'
    url = base_url + path
    response = requests.get(url, auth=auth, headers=headers, data=query)

    df = None
    try:  
        df = json_normalize(response.json()['hits']['hits'])
    except JSONDecodeError:
        print('@@ json not parseable', response)
    except Exception:
        print('@@ Exception! ', response)
    return df

In [5]:
def remove_self(stock_name, title_list):
    new_list = []
    for idx, title in enumerate(title_list):
        word, score = title
        if stock_name in word or \
          word in stock_name or \
          any(c.isdigit() for c in word) or \
          word in FILTERED_WORDS: \
            continue
        new_list.append((word, score))
    return new_list

In [6]:
FILTERED_WORDS = ['코스피', '내년', '오늘', '전날', '등이', '오른', '시장', '최대',
                  '최소', '따라', '이번', '지난', '정답', '주가', '결국', '마감', '목표가']
REMOVE_TARGET = ['JKB','JKG','JC','MAG']

def remove_josa(org):
    # mecab = Mecab()
    mecab = Mecab('usr/local/lib/mecab/dic/mecab-ko-dic')
    p_list = mecab.pos(org)
    p_list_cnt = len(p_list)
    tran_str = ''
    pointer = 0
    for c in org:
        p_word, p_type = p_list[pointer]
        p_next_word = ''
        p_next_type = None
        if pointer >= p_list_cnt:
            break
        elif pointer <= p_list_cnt - 2 :
            p_next_word, p_next_type = p_list[pointer+1]
        if c in [' ', '·']:
            tran_str += ' '
        if c in p_word:
            if p_type not in REMOVE_TARGET:
                tran_str += c
        elif c in p_next_word:
            if p_next_type not in REMOVE_TARGET:
                tran_str += c
            pointer += 1
        else:
            pointer += 1
    return tran_str

## Data

In [7]:
df_total = get_es_df('2022011711', '/poc_title_text/_search?size=10000')

query:  b'{"query": {"match": {"input_date": "2022011711"}}}'


In [8]:
df = df_total[df_total['_source.input_code'] == "SK"]
df.shape

(83, 18)

In [9]:
df.head(3)

,_index,_type,_id,_score,_source.page_url,_source.image_url,_source.crawl_dt,_source.title,_source.passed_time,_source.visible_txt,_source.agg_dt,_source.post_url,_source.indexed_at,_source.press,_source.etl_dt,_source.input_date,_source.task_type,_source.input_code
226,poc_title_text,_doc,c6bHZX4Bo-NoAHXjhVQB,4.745991,https://search.naver.com/search.naver?where=ne...,https://search.pstatic.net/common/?src=https%3...,2022-01-17 02:04:02.256,"메타버스 시대의 경제와 빈부격차, 그리고 그림자",1분 전,과거 SK커뮤니케이션이 싸이월드의 지속적인 운영에 실패하면서 이용자들이 구매했던 도...,20220117 11:04:41.565,http://www.ohmynews.com/NWS_Web/View/at_pg.asp...,2022-01-17T02:04:41.601145Z,오마이뉴스,1642417481510,2022011711,naver_news,SK
227,poc_title_text,_doc,dKbHZX4Bo-NoAHXjhVSD,4.745991,https://search.naver.com/search.naver?where=ne...,https://search.pstatic.net/common/?src=https%3...,2022-01-17 02:04:02.308,"코스피, 美 국채금리 급등·달러 강세 등 ‘외인·기관 쌍끌이 매도’",1분 전,"NAVER, 삼성SDI, SK이노베이션 등을 제외하면 시가총액 상위주가 모두 약세다...",20220117 11:04:41.692,http://www.inews24.com/view/1443365,2022-01-17T02:04:41.731251Z,아이뉴스24,1642417481510,2022011711,naver_news,SK
228,poc_title_text,_doc,dabHZX4Bo-NoAHXjhVS8,4.745991,https://search.naver.com/search.naver?where=ne...,https://search.pstatic.net/common/?src=https%3...,2022-01-17 02:04:02.411,"11번가-행복얼라이언스, 장애한부모가족에 도시락 지원",5분 전,SK의 이커머스 플랫폼 11번가의 사회환원 활동이 '11번가 희망쇼핑'을 통해 올해...,20220117 11:04:41.745,http://www.econovill.com/news/articleView.html...,2022-01-17T02:04:41.788667Z,이코노믹리뷰,1642417481510,2022011711,naver_news,SK


In [10]:
import re
stock_titles = df_total.groupby('_source.input_code')['_source.title'].apply(list)
text_list = stock_titles["SK"]
text_list = [" ".join(re.findall("[a-zA-Z가-힣0-9]+", title)) for title in text_list]
# text_list = [remove_josa(title) for title in text_list]
# print(">>> text_list : ", text_list)

In [11]:
# !pip3 install mecab-python3

In [12]:
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# Original KR-WordRank

In [13]:
min_count = 3 # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
wordrank_extractor = KRWordRank(min_count, max_length, verbose = False)
beta = 0.95 # PageRank의 decaying factor beta
max_iter = 20
keywords, rank, graph = wordrank_extractor.extract(text_list, beta, max_iter)

In [14]:
keywords

{'판독': 3.105789297530123,
 'SK': 3.0967792540120964,
 '민간': 3.0317894802497323,
 '기관': 3.010513197796919,
 '발간': 2.917464069990025,
 'AI': 2.882874800821029,
 '코스피': 2.80138038168747,
 '선임': 2.762224508454733,
 '가이드': 2.754729185873332,
 '전고체': 2.6497631454704322,
 '배터리': 2.6348541741067235,
 '개인정보보호': 2.5492060636478406,
 '그린랩스': 2.496377621471199,
 '솔루션': 2.295858981873046,
 '2900선': 2.1603590237743253,
 '붕괴': 2.1268741217600855,
 '전문가': 2.0446090183792984,
 '장중': 2.0126711159072936,
 '1700억': 1.7876878253066197,
 '승소': 1.6583491705351345,
 '개발': 1.5541663164597412,
 '에너지': 1.4923692987054995,
 '경쟁': 1.4503404485474183,
 '뇌출': 1.388632909649923,
 '매도에': 1.3305581825026547,
 '하락': 1.3279498270220356,
 '외인': 1.3076755588025597,
 '건설': 1.2943018951894365,
 '철퇴': 1.1795921505319154,
 '협력': 1.1540206362477512,
 '다시': 1.13250355817495,
 '형광': 1.1109951154247286,
 '외국인': 1.081359413068033,
 '사외이사': 1.0794295151914923,
 '평창': 1.0736043571588727,
 '투자': 1.0614850698666618,
 '교수': 1.0,
 'LG': 

In [246]:
wordrank_result

[('판독', 3.105789297530123),
 ('민간', 3.0317894802497323),
 ('기관', 3.010513197796919),
 ('발간', 2.917464069990025),
 ('AI', 2.882874800821029),
 ('선임', 2.762224508454733),
 ('가이드', 2.754729185873332),
 ('전고체', 2.6497631454704322),
 ('배터리', 2.6348541741067235),
 ('개인정보보호', 2.5492060636478406),
 ('그린랩스', 2.496377621471199),
 ('솔루션', 2.295858981873046)]

In [15]:
rank

{11: 2.1153937299659535,
 52: 1.2708344871949668,
 10: 2.80138038168747,
 184: 0.9398463132925592,
 114: 2.1268741217600855,
 7: 1.7709348519991295,
 179: 2.0126711159072936,
 118: 1.081359413068033,
 19: 3.010513197796919,
 75: 1.3076755588025597,
 41: 2.1603590237743253,
 32: 1.0246972625319875,
 12: 2.1153937299659535,
 8: 1.7709348519991295,
 213: 0.25950135182305917,
 273: 0.6383417763202134,
 292: 0.30551044817591355,
 319: 1.1540206362477512,
 2: 3.783101633741767,
 74: 0.6605762554978297,
 18: 1.92307713844631,
 119: 0.8936255713953782,
 160: 1.3279498270220356,
 323: 0.4507615903078749,
 17: 1.7844048091419824,
 120: 0.8344761685873682,
 148: 1.3305581825026547,
 9: 2.5203976894375257,
 21: 1.0812827024678089,
 83: 0.9699556854538021,
 279: 0.5247031331592387,
 5: 2.529934586044356,
 217: 0.5613260018093043,
 218: 1.2943018951894365,
 282: 0.5879175839930658,
 1: 3.0967792540120964,
 25: 0.630828219138878,
 86: 1.0239486592103304,
 53: 1.3080346680150567,
 3: 1.727495628546075

In [16]:
graph

{11: {7: 0.7142857142857143,
  75: 0.07142857142857142,
  19: 0.08823529411764706,
  41: 0.045454545454545456,
  32: 0.09090909090909091,
  179: 0.043478260869565216,
  118: 0.25},
 52: {7: 0.047619047619047616,
  8: 0.047619047619047616,
  75: 0.14285714285714285,
  19: 0.08823529411764706,
  6: 0.20833333333333334,
  83: 0.1,
  282: 0.16666666666666666},
 10: {7: 0.047619047619047616,
  8: 0.047619047619047616,
  17: 0.2,
  5: 0.1111111111111111,
  25: 0.3333333333333333,
  100: 0.14285714285714285,
  14: 0.03333333333333333,
  43: 0.058823529411764705,
  44: 0.058823529411764705,
  45: 0.058823529411764705,
  78: 0.14285714285714285,
  32: 0.09090909090909091,
  116: 0.1,
  15: 0.14285714285714285,
  85: 0.5,
  120: 0.3333333333333333,
  301: 0.25,
  63: 0.25},
 184: {7: 0.14285714285714285,
  8: 0.14285714285714285,
  116: 0.3,
  322: 0.3333333333333333},
 114: {7: 0.047619047619047616,
  8: 0.047619047619047616,
  17: 0.05,
  42: 0.3076923076923077,
  29: 0.3333333333333333,
  30:

# Tunend KR-WordRank

In [17]:
from collections import defaultdict
import math
import numpy as np

from krwordrank.graph import hits


def summarize_with_keywords(texts, num_keywords=100, stopwords=None, min_count=5,
    max_length=10, beta=0.85, max_iter=10, num_rset=-1, verbose=False):
    """
    It train KR-WordRank to extract keywords from texts.

        >>> from krwordrank.word import summarize_with_keywords

        >>> texts = [] # list of str
        >>> keywords = summarize_with_keywords(texts, num_keywords=100, min_count=5)

    Arguments
    ---------
    texts : list of str
        Each str is a sentence.
    num_keywords : int
        Number of keywords extracted from KR-WordRank
        Default is 100.
    stopwords : None or set of str
        Stopwords list for keyword and key-sentence extraction
    min_count : int
        Minimum frequency of subwords used to construct subword graph
        Default is 5
    max_length : int
        Maximum length of subwords used to construct subword graph
        Default is 10
    beta : float
        PageRank damping factor. 0 < beta < 1
        Default is 0.85
    max_iter : int
        Maximum number of iterations of HITS algorithm.
        Default is 10
    num_rset : int
        Number of R set words sorted by rank. It will be used to L-part word filtering.
        Default is -1.
    verbose : Boolean
        If True, it shows training status
        Default is False

    Returns
    -------
    keywords : dict
        Word : rank dictionary. keywords[str] = float

    Usage
    -----
        >>> from krwordrank.word import summarize_with_keywords

        >>> texts = [] # list of str
        >>> keywords = summarize_with_keywords(texts, num_keywords=100, min_count=5)
    """
    # train KR-WordRank
    wordrank_extractor = KRWordRank(
        min_count = min_count,
        max_length = max_length,
        verbose = verbose
        )

    keywords, rank, graph = wordrank_extractor.extract(texts,
        beta, max_iter, num_rset=num_rset)

    # stopword filtering
    if stopwords is None:
        stopwords = {}
    keywords = {word:r for word, r in keywords.items() if not (word in stopwords)}

    # top rank filtering
    if num_keywords > 0:
        keywords = {word:r for word, r in sorted(keywords.items(), key=lambda x:-x[1])[:num_keywords]}

    return keywords


class KRWordRank_test:
    """Unsupervised Korean Keyword Extractor

    Implementation of Kim, H. J., Cho, S., & Kang, P. (2014). KR-WordRank: 
    An Unsupervised Korean Word Extraction Method Based on WordRank. 
    Journal of Korean Institute of Industrial Engineers, 40(1), 18-33.

    Arguments
    ---------
    min_count : int
        Minimum frequency of subwords used to construct subword graph
        Default is 5
    max_length : int
        Maximum length of subwords used to construct subword graph
        Default is 10
    verbose : Boolean
        If True, it shows training status
        Default is False

    Usage
    -----
        >>> from krwordrank.word import KRWordRank

        >>> texts = ['예시 문장 입니다', '여러 문장의 list of str 입니다', ... ]
        >>> wordrank_extractor = KRWordRank()
        >>> keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter, verbose)
    """
    def __init__(self, min_count=5, max_length=10, verbose=False):
        self.min_count = min_count
        self.max_length = max_length
        self.verbose = verbose
        self.sum_weight = 1
        self.vocabulary = {}
        self.index2vocab = []

    def scan_vocabs(self, docs):
        """
        It scans subwords positioned of left-side (L) and right-side (R) of words.
        After scanning was done, KR-WordRank has index2vocab as class attribute.

        Arguments
        ---------
        docs : list of str
            Sentence list

        Returns
        -------
        counter : dict
            {(subword, 'L')] : frequency}
        """
        self.vocabulary = {}
        if self.verbose:
            print('scan vocabs ... ')

        counter = {}
        for doc in docs:

            for token in doc.split():
                len_token = len(token)
                counter[(token, 'L')] = counter.get((token, 'L'), 0) + 1

                for e in range(1, min(len(token), self.max_length)):
                    if (len_token - e) > self.max_length:
                        continue

                    l_sub = (token[:e], 'L')
                    r_sub = (token[e:], 'R')
                    counter[l_sub] = counter.get(l_sub, 0) + 1
                    counter[r_sub] = counter.get(r_sub, 0) + 1

        counter = {token:freq for token, freq in counter.items() if freq >= self.min_count}
        for token, _ in sorted(counter.items(), key=lambda x:x[1], reverse=True):
            self.vocabulary[token] = len(self.vocabulary)

        self._build_index2vocab()

        if self.verbose:
            print('num vocabs = %d' % len(counter))
        return counter

    def _build_index2vocab(self):
        self.index2vocab = [vocab for vocab, index in sorted(self.vocabulary.items(), key=lambda x:x[1])]
        self.sum_weight = len(self.index2vocab)
    
    def extract(self, docs, beta=0.85, max_iter=10, num_keywords=-1,
        num_rset=-1, vocabulary=None, bias=None, rset=None):
        """
        It constructs word graph and trains ranks of each node using HITS algorithm.
        After training it selects suitable subwords as words.

        Arguments
        ---------
        docs : list of str
            Sentence list.
        beta : float
            PageRank damping factor. 0 < beta < 1
            Default is 0.85
        max_iter : int
            Maximum number of iterations of HITS algorithm.
            Default is 10
        num_keywords : int
            Number of keywords sorted by rank.
            Default is -1. If the vaule is negative, it returns all extracted words.
        num_rset : int
            Number of R set words sorted by rank. It will be used to L-part word filtering.
            Default is -1.
        vocabulary : None or dict
            User specified vocabulary to index mapper
        bias : None or dict
            User specified HITS bias term
        rset : None or dict
            User specfied R set

        Returns
        -------
        keywords : dict
            word : rank dictionary. {str:float}
        rank : dict
            subword : rank dictionary. {int:float}
        graph : dict of dict
            Adjacent subword graph. {int:{int:float}}

        Usage
        -----
            >>> from krwordrank.word import KRWordRank

            >>> texts = ['예시 문장 입니다', '여러 문장의 list of str 입니다', ... ]
            >>> wordrank_extractor = KRWordRank()
            >>> keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter, verbose)
        """

        rank, graph = self.train(docs, beta, max_iter, vocabulary, bias)
        # print(rank)
        # print(graph)
        lset = {self.int2token(idx)[0]:r for idx, r in rank.items() if self.int2token(idx)[1] == 'L'}
        if not rset:
            rset = {self.int2token(idx)[0]:r for idx, r in rank.items() if self.int2token(idx)[1] == 'R'}

        if num_rset > 0:
            rset = {token:r for token, r in sorted(rset.items(), key=lambda x:-x[1])[:num_rset]}

        keywords = self._select_keywords(lset, rset)
        # print(keywords)
        keywords = self._filter_compounds(keywords)
        # print(keywords)
        keywords = self._filter_subtokens(keywords)
        # print(keywords)
        if num_keywords > 0:
            keywords = {token:r for token, r in sorted(keywords.items(), key=lambda x:-x[1])[:num_keywords]}
        # print(self.vocabulary)
        return self.vocabulary, keywords, rank, graph

    def _select_keywords(self, lset, rset):
        keywords = {}
        for word, r in sorted(lset.items(), key=lambda x:x[1], reverse=True):
            len_word = len(word)
            if len_word == 1:
                continue

            is_compound = False
            for e in range(2, len_word):
                if (word[:e] in keywords) and (word[:e] in rset):
                    is_compound = True
                    break

            if not is_compound:
                keywords[word] = r

        return keywords

    def _filter_compounds(self, keywords):
        keywords_= {}
        for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True):
            len_word = len(word)

            if len_word <= 2:
                keywords_[word] = r
                continue

            if len_word == 3:
                if word[:2] in keywords_:
                    continue

            is_compound = False
            for e in range(2, len_word - 1):
                # fixed. comment from Y. cho
                if (word[:e] in keywords) and (word[e:] in keywords):
                    is_compound = True
                    break

            if not is_compound:
                keywords_[word] = r

        return keywords_

    def _filter_subtokens(self, keywords):
        subtokens = set()
        keywords_ = {}

        for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True):
            subs = {word[:e] for e in range(2, len(word)+1)}

            is_subtoken = False
            for sub in subs:
                if sub in subtokens:
                    is_subtoken = True
                    break

            if not is_subtoken:
                keywords_[word] = r
                subtokens.update(subs)

        return keywords_

    def train(self, docs, beta=0.85, max_iter=10, vocabulary=None, bias=None):
        """
        It constructs word graph and trains ranks of each node using HITS algorithm.
        Use this function only when you want to train rank of subwords

        Arguments
        ---------
        docs : list of str
            Sentence list.
        beta : float
            PageRank damping factor. 0 < beta < 1
            Default is 0.85
        max_iter : int
            Maximum number of iterations of HITS algorithm.
            Default is 10
        vocabulary : None or dict
            User specified vocabulary to index mapper
        bias : None or dict
            User specified HITS bias term
            {str: float} Format

        Returns
        -------
        rank : dict
            subword : rank dictionary. {int:float}
        graph : dict of dict
            Adjacent subword graph. {int:{int:float}}
        """
        # print(vocabulary)
        if (not vocabulary) and (not self.vocabulary):
            self.scan_vocabs(docs)
        elif (not vocabulary):
            self.vocabulary = vocabulary
            self._build_index2vocab()
        
        graph = self._construct_word_graph(docs)
        # print(graph)
        # # add custom bias dict
        encoded_bias = {}
        custom_bias_dict = bias

        if custom_bias_dict:
            for word, value in custom_bias_dict.items():
                encoded_word = self.token2int((word, 'L'))
                if encoded_word != -1:
                    encoded_bias[encoded_word] = value

        rank = hits(graph, beta, max_iter, encoded_bias,
                    sum_weight=self.sum_weight,
                    number_of_nodes=len(self.vocabulary),
                    verbose=self.verbose
                    )

        return rank, graph

    def token2int(self, token):
        """
        Arguments
        ---------
        token : tuple
            (subword, 'L') or (subword, 'R')
            For example, ('이것', 'L') or ('은', 'R')

        Returns
        -------
        index : int
            Corresponding index
            If it is unknown, it returns -1
        """
        return self.vocabulary.get(token, -1)

    def int2token(self, index):
        """
        Arguments
        ---------
        index : int
            Token index

        Returns
        -------
        token : tuple
            Corresponding index formed such as (subword, 'L') or (subword, 'R')
            For example, ('이것', 'L') or ('은', 'R').
            If it is unknown, it returns None
        """
        return self.index2vocab[index] if (0 <= index < len(self.index2vocab)) else None

    def _construct_word_graph(self, docs):
        def normalize(graph):
            graph_ = defaultdict(lambda: defaultdict(lambda: 0))
            # print(graph)
            for from_, to_dict in graph.items():
                # print("from_ : ", from_)
                # print("to_dict : ", to_dict)
                sum_ = sum(to_dict.values())
                # print("sum_ : ", to_dict)
                for to_, w in to_dict.items():
                    # print("to_ : ", to_)
                    # print("w : ", w)
                    # print("w : ", w)
                    graph_[to_][from_] = w / sum_
            graph_ = {t:dict(fd) for t, fd in graph_.items()}
            # print(graph_)
            return graph_

        graph = defaultdict(lambda: defaultdict(lambda: 0))
        # print(graph)
        # print(docs)
        for doc in docs:

            tokens = doc.split()
            # print(tokens)
            # print("==========")
            if not tokens:
                continue

            links = []
            for token in tokens:
                links += self._intra_link(token)
                # print(links)
                # print("=====")
            if len(tokens) > 1:
                tokens = [tokens[-1]] + tokens + [tokens[0]]
                links += self._inter_link(tokens)

            links = self._check_token(links)
            # print(links)
            if not links:
                continue

            links = self._encode_token(links)
            # print(links)
            for l_node, r_node in links:
                graph[l_node][r_node] += 1
                graph[r_node][l_node] += 1
        # print(graph)
        # reverse for inbound graph. but it normalized with sum of outbound weight
        graph = normalize(graph)
        # print(graph)
        return graph

    def _intra_link(self, token):
        links = []
        len_token = len(token)
        for e in range(1, min(len_token, 10)):
            if (len_token - e) > self.max_length:
                continue
            links.append( ((token[:e], 'L'), (token[e:], 'R')) )
        return links

    def _inter_link(self, tokens):
        def rsub_to_token(t_left, t_curr):
            return [((t_left[-b:], 'R'), (t_curr, 'L')) for b in range(1, min(10, len(t_left)))]
        def token_to_lsub(t_curr, t_rigt):
            return [((t_curr, 'L'), (t_rigt[:e], 'L')) for e in range(1, min(10, len(t_rigt)))]

        links = []
        for i in range(1, len(tokens)-1):
            links += rsub_to_token(tokens[i-1], tokens[i])
            links += token_to_lsub(tokens[i], tokens[i+1])
        return links

    def _check_token(self, token_list):
        # print(token_list)
        return [(token[0], token[1]) for token in token_list if (token[0] in self.vocabulary and token[1] in self.vocabulary)]

    def _encode_token(self, token_list):
        # print(token_list)
        # print("============")
        # print([(self.vocabulary[token[0]],self.vocabulary[token[1]]) for token in token_list])
        # print(self.vocabulary)
        return [(self.vocabulary[token[0]],self.vocabulary[token[1]]) for token in token_list]

In [18]:
min_count = 3 # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
wordrank_extractor = KRWordRank_test(min_count, max_length, verbose=False)
beta = 0.95 # PageRank의 decaying factor beta
max_iter = 20
vocabulary, keywords, rank, graph = wordrank_extractor.extract(text_list, beta, max_iter)

In [19]:
vocabulary

{('S', 'L'): 0,
 ('SK', 'L'): 1,
 ('C', 'L'): 2,
 ('1', 'L'): 3,
 ('스', 'R'): 4,
 ('기', 'L'): 5,
 ('에', 'R'): 6,
 ('코', 'L'): 7,
 ('코스', 'L'): 8,
 ('관', 'R'): 9,
 ('코스피', 'L'): 10,
 ('스피', 'R'): 11,
 ('피', 'R'): 12,
 ('가', 'R'): 13,
 ('2', 'L'): 14,
 ('시', 'L'): 15,
 ('K', 'R'): 16,
 ('외', 'L'): 17,
 ('인', 'R'): 18,
 ('기관', 'L'): 19,
 ('선', 'R'): 20,
 ('대', 'R'): 21,
 ('리', 'R'): 22,
 ('가', 'L'): 23,
 ('션', 'R'): 24,
 ('도', 'R'): 25,
 ('전', 'L'): 26,
 ('배', 'L'): 27,
 ('개', 'L'): 28,
 ('00선', 'R'): 29,
 ('0선', 'R'): 30,
 ('드', 'R'): 31,
 ('장', 'L'): 32,
 ('원', 'R'): 33,
 ('주', 'L'): 34,
 ('소', 'R'): 35,
 ('사', 'R'): 36,
 ('배터리', 'L'): 37,
 ('터리', 'R'): 38,
 ('배터', 'L'): 39,
 ('는', 'R'): 40,
 ('2900선', 'L'): 41,
 ('900선', 'R'): 42,
 ('29', 'L'): 43,
 ('290', 'L'): 44,
 ('2900', 'L'): 45,
 ('간', 'R'): 46,
 ('이', 'L'): 47,
 ('AI', 'L'): 48,
 ('A', 'L'): 49,
 ('호', 'R'): 50,
 ('루션', 'R'): 51,
 ('매도', 'L'): 52,
 ('매', 'L'): 53,
 ('대', 'L'): 54,
 ('I', 'R'): 55,
 ('뇌출혈', 'L'): 56,
 ('뇌', 'L'

In [133]:
values = vocabulary.values()
values_list = list(values)
keys = vocabulary.keys()
keys_list = list(keys)

a_value = values_list[0]
# print(values_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [192]:
keys_list[0][0]

'S'

In [153]:
vocabulary

{('S', 'L'): 0,
 ('SK', 'L'): 1,
 ('C', 'L'): 2,
 ('1', 'L'): 3,
 ('스', 'R'): 4,
 ('기', 'L'): 5,
 ('에', 'R'): 6,
 ('코', 'L'): 7,
 ('코스', 'L'): 8,
 ('관', 'R'): 9,
 ('코스피', 'L'): 10,
 ('스피', 'R'): 11,
 ('피', 'R'): 12,
 ('가', 'R'): 13,
 ('2', 'L'): 14,
 ('시', 'L'): 15,
 ('K', 'R'): 16,
 ('외', 'L'): 17,
 ('인', 'R'): 18,
 ('기관', 'L'): 19,
 ('선', 'R'): 20,
 ('대', 'R'): 21,
 ('리', 'R'): 22,
 ('가', 'L'): 23,
 ('션', 'R'): 24,
 ('도', 'R'): 25,
 ('전', 'L'): 26,
 ('배', 'L'): 27,
 ('개', 'L'): 28,
 ('00선', 'R'): 29,
 ('0선', 'R'): 30,
 ('드', 'R'): 31,
 ('장', 'L'): 32,
 ('원', 'R'): 33,
 ('주', 'L'): 34,
 ('소', 'R'): 35,
 ('사', 'R'): 36,
 ('배터리', 'L'): 37,
 ('터리', 'R'): 38,
 ('배터', 'L'): 39,
 ('는', 'R'): 40,
 ('2900선', 'L'): 41,
 ('900선', 'R'): 42,
 ('29', 'L'): 43,
 ('290', 'L'): 44,
 ('2900', 'L'): 45,
 ('간', 'R'): 46,
 ('이', 'L'): 47,
 ('AI', 'L'): 48,
 ('A', 'L'): 49,
 ('호', 'R'): 50,
 ('루션', 'R'): 51,
 ('매도', 'L'): 52,
 ('매', 'L'): 53,
 ('대', 'L'): 54,
 ('I', 'R'): 55,
 ('뇌출혈', 'L'): 56,
 ('뇌', 'L'

In [25]:
vocab_dict = dict([(value, key[0]) for key, value in vocabulary.items()])
vocab_dict

{0: 'S',
 1: 'SK',
 2: 'C',
 3: '1',
 4: '스',
 5: '기',
 6: '에',
 7: '코',
 8: '코스',
 9: '관',
 10: '코스피',
 11: '스피',
 12: '피',
 13: '가',
 14: '2',
 15: '시',
 16: 'K',
 17: '외',
 18: '인',
 19: '기관',
 20: '선',
 21: '대',
 22: '리',
 23: '가',
 24: '션',
 25: '도',
 26: '전',
 27: '배',
 28: '개',
 29: '00선',
 30: '0선',
 31: '드',
 32: '장',
 33: '원',
 34: '주',
 35: '소',
 36: '사',
 37: '배터리',
 38: '터리',
 39: '배터',
 40: '는',
 41: '2900선',
 42: '900선',
 43: '29',
 44: '290',
 45: '2900',
 46: '간',
 47: '이',
 48: 'AI',
 49: 'A',
 50: '호',
 51: '루션',
 52: '매도',
 53: '매',
 54: '대',
 55: 'I',
 56: '뇌출혈',
 57: '뇌',
 58: '출혈',
 59: '뇌출',
 60: '혈',
 61: '시',
 62: '와',
 63: '자',
 64: '손',
 65: '하',
 66: '한',
 67: '평',
 68: '평창',
 69: '다',
 70: '신',
 71: '경',
 72: '그',
 73: '고',
 74: '강',
 75: '외인',
 76: '락',
 77: '조',
 78: '사',
 79: '선',
 80: '판독',
 81: '판',
 82: '독',
 83: '반',
 84: '수',
 85: '황',
 86: '도에',
 87: '17',
 88: '억',
 89: '개인',
 90: '개인정',
 91: '개인정보',
 92: '발',
 93: 'L',
 94: 'LG',
 95: '상',
 96: 

In [28]:
graph_token = []
for key1, value1 in graph.items():
    tmp = []
    for key2, value2 in value1.items():
        tmp.append((vocab_dict.get(key2), value2))
    graph_token.append((vocab_dict.get(key1), dict(tmp)))
graph_token = dict(graph_token)    

In [29]:
dict(graph_token)

{'스피': {'코': 0.7142857142857143,
  '외인': 0.07142857142857142,
  '기관': 0.08823529411764706,
  '2900선': 0.045454545454545456,
  '장': 0.09090909090909091,
  '장중': 0.043478260869565216,
  '외국인': 0.25},
 '매도': {'코': 0.047619047619047616,
  '코스': 0.047619047619047616,
  '외인': 0.14285714285714285,
  '기관': 0.08823529411764706,
  '에': 0.20833333333333334,
  '반': 0.1,
  '동반': 0.16666666666666666},
 '코스피': {'코': 0.047619047619047616,
  '코스': 0.047619047619047616,
  '외': 0.2,
  '기': 0.1111111111111111,
  '도': 0.3333333333333333,
  '국': 0.14285714285714285,
  '2': 0.03333333333333333,
  '29': 0.058823529411764705,
  '290': 0.058823529411764705,
  '2900': 0.058823529411764705,
  '사': 0.14285714285714285,
  '장': 0.09090909090909091,
  '괴': 0.1,
  '시': 0.14285714285714285,
  '황': 0.5,
  '외국': 0.3333333333333333,
  '회': 0.25,
  '자': 0.25},
 '시황': {'코': 0.14285714285714285,
  '코스': 0.14285714285714285,
  '괴': 0.3,
  '전': 0.3333333333333333},
 '붕괴': {'코': 0.047619047619047616,
  '코스': 0.04761904761904761